# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory
Data_URL= 'https://github.com/jfcarmonag/nd00333-capstone/raw/master/starter_file/BankChurners.csv'
ds=TabularDatasetFactory.from_delimited_files(path=Data_URL)

ws = Workspace.from_config()
experiment_name = 'exp-capstone-hyperdrive'

experiment=Experiment(ws, experiment_name)

cluster_name = 'project-cluster'
try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
##ONLY RUN FOR LOGISTIC REGRESSION

from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal

# Specify parameter sampler


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3,slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--max_depth': choice(3,4,5),
                 '--learning_rate':uniform(0.05,0.1),
                 '--gamma': choice(0.0,0.25,0.5,1.0),
                 '--reg_lambda':choice(0,1.0,10.0),
                 '--scale_pos_weight': choice (1,3,5)})

#TODO: Create your estimator and hyperdrive config
estimator = Estimator(
    source_directory="./",
    entry_script="train.py",
    compute_target=compute_target,
)

hyperdrive_run_config = HyperDriveConfig(
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50,
    max_concurrent_runs=8,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    estimator=estimator
)

'Estimator' is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or an Azure ML curated environment.


In [4]:
##RUN FOR XGBOOST

from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.train.sklearn import SKLearn
from azureml.train.estimator import Estimator
from azureml.core import 
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal



# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3,slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--max_depth': choice(3,4,5),
                 '--learning_rate':uniform(0.05,0.1),
                 '--gamma': choice(0.0,0.25,0.5,1.0),
                 '--reg_lambda':choice(0,1.0,10.0),
                 '--scale_pos_weight': choice (1,3,5)})

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(
    source_directory="./",
    entry_script="train.py",
    compute_target=compute_target,
)

hyperdrive_run_config = HyperDriveConfig(
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50,
    max_concurrent_runs=8,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    run_config=estimator
)

NameError: name 'ScriptRunConfig' is not defined

In [ ]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# register model 
model = best_run.register_model(model_name='model', model_path='outputs/model.pkl')
print(f"Model {model.name}.v{model.version} correctly saved")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import xgboost as xgb 
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error, confusion_matrix
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
_DATA_URL = 'https://github.com/jfcarmonag/nd00333-capstone/raw/master/starter_file/BankChurners.csv'

def clean_data(data):
    
    # Clean and one hot encode data
    # Cleaning and one-hot encoding extracted from 
    # https://www.kaggle.com/sakshigoyal7/churned-customers-recall-of-86
    x_df = data.to_pandas_dataframe().dropna()
    x_df['Attrition_Flag'].replace({'Existing Customer':0, 
    'Attrited Customer':1},inplace=True)
    x_df.drop(x_df.columns[[0,-1,-2]].values,axis=1,inplace=True)
    map_education_level = {'High School':1,'Graduate':3,'Uneducated':0,
    'College':2,'Post-Graduate':4,'Doctorate':5, 'Unknown':0}
    map_income_level = {'$60K - $80K':3,'Less than $40K':1, '$80K - $120K':4,
    '$40K - $60K':2,'$120K +':5, 'Unknown':0}
    map_card_category = {'Blue':1,'Gold':3,'Silver':2,'Platinum':4}
    x_df['Education_Level'].replace(map_education_level,inplace=True)
    x_df['Income_Category'].replace(map_income_level,inplace=True)
    x_df['Card_Category'].replace(map_card_category,inplace=True)
    # #hot encoding of gender category
    x_df.insert(2,'Gender_M',x_df['Gender'],True)
    x_df.rename({'Gender':'Gender_F'},axis=1,inplace=True)
    x_df['Gender_M'].replace({'M':1,'F':0},inplace=True)
    x_df['Gender_F'].replace({'M':0,'F':1},inplace=True)
    #
    # #hot encoding of marital status
    x_df.insert(7,'Single',x_df['Marital_Status'],True)
    x_df.insert(7,'Divorced',x_df['Marital_Status'],True)
    x_df.insert(7,'Unknown',x_df['Marital_Status'],True)
    x_df.rename({'Marital_Status':'Married'},axis=1,inplace=True)
    x_df['Married'].replace({'Single':0, 'Married':1, 'Divorced':0, 'Unknown':0},
    inplace=True)
    x_df['Single'].replace({'Single':1, 'Married':0, 'Divorced':0, 'Unknown':0},
    inplace=True)
    x_df['Divorced'].replace({'Single':0, 'Married':0, 'Divorced':1, 'Unknown':0},
    inplace=True)
    x_df['Unknown'].replace({'Single':0, 'Married':0, 'Divorced':0, 'Unknown':1},
    inplace=True)
    y_df = x_df.pop("Attrition_Flag")
    return x_df, y_df


def get_clean_data():
    ds = TabularDatasetFactory.from_delimited_files(path=_DATA_URL)
    return clean_data(ds)


def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    # parser.add_argument('--C', type=float, default=1.0, 
    # help="Inverse of regularization strength. 
    # Smaller values cause stronger regularization")
    # parser.add_argument('--max_iter', type=int, default=100, 
    # help="Maximum number of iterations to converge")
    parser.add_argument('--max_depth', type=float, default= 4)
    parser.add_argument('--learning_rate', type=float, default= 0.1)
    parser.add_argument('--gamma', type=float, default= 0.5)
    parser.add_argument('--reg_lambda', type=float, default= 1.0)
    parser.add_argument('--scale_pos_weight', type=float, default=3.0)
    args, unknown = parser.parse_known_args()

    run = Run.get_context()

    run.log("max_depth:", np.int(args.max_depth))
    run.log("learning_rate:", np.float(args.learning_rate))
    run.log("gamma:", np.float(args.gamma))
    run.log("reg_lambda:", np.float(args.reg_lambda))
    run.log("scale_pos_weight:", np.float(args.scale_pos_weight))

    x, y = get_clean_data()
    print(set(x['Income_Category']))
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, 
                                                        random_state=42)
    model = xgb.XGBClassifier(objective="binary:logistic", 
                            missing=None,
                            max_depth=args.max_depth, 
                            learning_rate=args.learning_rate, 
                            gamma=args.gamma, 
                            reg_lambda=args.reg_lambda,
                            scale_pos_weight=args.scale_pos_weight)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_pred, y_test)
    run.log("Accuracy", np.float(accuracy))
    #value = {
    #   "schema_type": "confusion_matrix",
    #   "schema_version": "v1",
    #   "data": {
    #       "class_labels": ["0", "1"],
    #       "matrix": confusion_matrix(y_test, model.predict(x_test)).tolist()
    #   }
    #}
    #run.log_confusion_matrix(name='Confusion Matrix', value=value)
    os.makedirs('outputs', exist_ok=True)
    # note file saved in the outputs folder is automatically uploaded 
    # into experiment record
    joblib.dump(value=model, filename='outputs/model.pkl')

if __name__ == '__main__':
    main()
